In [1]:
import requests

In [2]:
#Option A — Use asyncio + aiohttp (10× faster)
# Allows Jupyter/VSCode notebooks to reuse the existing event loop.
# Without this, asyncio.run() would fail because Jupyter already runs a loop.
import nest_asyncio
nest_asyncio.apply()

import asyncio
import aiohttp
import pandas as pd


# ------------------------------------------------------------
# FUNCTION: get_all_pokemon_urls
# ------------------------------------------------------------
# This function retrieves the list of ALL Pokémon from the PokeAPI.
# The endpoint returns basic info: name + URL for each Pokémon.
# We only extract the URLs because each URL points to the full data
# for that specific Pokémon (stats, abilities, types, etc.).
# ------------------------------------------------------------
async def get_all_pokemon_urls(session):
    url = "https://pokeapi.co/api/v2/pokemon?limit=2000"

    # Make an asynchronous GET request using the shared session.
    async with session.get(url) as response:
        data = await response.json()  # Convert JSON → Python dict

        # Extract only the "url" field for each Pokémon.
        # Example: "https://pokeapi.co/api/v2/pokemon/25/"
        return [pokemon["url"] for pokemon in data["results"]]


# ------------------------------------------------------------
# FUNCTION: fetch_pokemon
# ------------------------------------------------------------
# This function retrieves the FULL details for a single Pokémon.
# It is called many times (once per Pokémon), but asyncio allows
# all of these requests to run concurrently, making it extremely fast.
#
# The function extracts:
# - id
# - name
# - height
# - weight
# - base experience
# - list of types
# - list of abilities
# - stats (HP, attack, defense, etc.) as a dictionary
# ------------------------------------------------------------
async def fetch_pokemon(session, url):
    async with session.get(url) as response:
        if response.status == 200:
            data = await response.json()

            return {
                "id": data["id"],
                "name": data["name"],
                "height": data["height"],
                "weight": data["weight"],
                "base_experience": data["base_experience"],

                # Extract Pokémon types (e.g., ["electric"])
                "types": [t["type"]["name"] for t in data["types"]],

                # Extract abilities (e.g., ["static", "lightning-rod"])
                "abilities": [a["ability"]["name"] for a in data["abilities"]],

                # Extract stats into a dictionary:
                # {"hp": 35, "attack": 55, "defense": 40, ...}
                "stats": {s["stat"]["name"]: s["base_stat"] for s in data["stats"]}
            }


# ------------------------------------------------------------
# FUNCTION: main
# ------------------------------------------------------------
# This is the main asynchronous workflow:
# 1. Create a single aiohttp session (efficient for many requests)
# 2. Fetch all Pokémon URLs
# 3. Create a list of async tasks (one per Pokémon)
# 4. Run all tasks concurrently using asyncio.gather()
# 5. Filter out any failed requests
# 6. Convert the results into a pandas DataFrame
# ------------------------------------------------------------
async def main():
    async with aiohttp.ClientSession() as session:

        # Step 1: Get URLs for all Pokémon
        urls = await get_all_pokemon_urls(session)

        # Step 2: Create async tasks for each Pokémon
        tasks = [fetch_pokemon(session, url) for url in urls]

        # Step 3: Run all tasks concurrently (this is the speed boost)
        results = await asyncio.gather(*tasks)

        # Step 4: Remove any None results (failed requests)
        results = [r for r in results if r is not None]

        # Step 5: Convert the list of dictionaries into a DataFrame
        df = pd.DataFrame(results)
        return df


# ------------------------------------------------------------
# EXECUTION (Jupyter-friendly)
# ------------------------------------------------------------
# In Jupyter, we cannot use asyncio.run(), so we use "await" directly.
# This triggers the asynchronous workflow and returns the DataFrame.
# ------------------------------------------------------------
df = await main()

# Display the final table of ALL Pokémon
df


,id,name,height,weight,base_experience,types,abilities,stats
0,1,bulbasaur,7,69,64.0,"[grass, poison]","[overgrow, chlorophyll]","{'hp': 45, 'attack': 49, 'defense': 49, 'speci..."
1,2,ivysaur,10,130,142.0,"[grass, poison]","[overgrow, chlorophyll]","{'hp': 60, 'attack': 62, 'defense': 63, 'speci..."
2,3,venusaur,20,1000,236.0,"[grass, poison]","[overgrow, chlorophyll]","{'hp': 80, 'attack': 82, 'defense': 83, 'speci..."
3,4,charmander,6,85,62.0,[fire],"[blaze, solar-power]","{'hp': 39, 'attack': 52, 'defense': 43, 'speci..."
4,5,charmeleon,11,190,142.0,[fire],"[blaze, solar-power]","{'hp': 58, 'attack': 64, 'defense': 58, 'speci..."
...,...,...,...,...,...,...,...,...
1345,10321,glimmora-mega,28,770,NaN,"[rock, poison]",[],"{'hp': 83, 'attack': 90, 'defense': 105, 'spec..."
1346,10322,tatsugiri-curly-mega,6,240,NaN,"[dragon, water]",[],"{'hp': 68, 'attack': 65, 'defense': 90, 'speci..."
1347,10323,tatsugiri-droopy-mega,6,240,NaN,"[dragon, water]",[],"{'hp': 68, 'attack': 65, 'defense': 90, 'speci..."
1348,10324,tatsugiri-stretchy-mega,6,240,NaN,"[dragon, water]",[],"{'hp': 68, 'attack': 65, 'defense': 90, 'speci..."


In [ ]:
import random

def generate_question(df):
    """
    Generates a single multiple‑choice quiz question about a random Pokémon.
    The question can be about:
      - one of its types
      - one of its abilities
      - one of its base stats (HP, attack, etc.)

    Returns:
      question (str): the question text
      options (list): list of 4 answer choices (shuffled)
      correct (any): the correct answer value
    """

    # ------------------------------------------------------------
    # 1. Pick a random Pokémon from the DataFrame
    # ------------------------------------------------------------
    # df.sample(1) returns a random row; iloc[0] extracts it as a Series.
    pokemon = df.sample(1).iloc[0]

    # ------------------------------------------------------------
    # 2. Randomly choose what type of question to ask
    # ------------------------------------------------------------
    question_type = random.choice(["type", "ability", "stat"])

    # ------------------------------------------------------------
    # 3A. TYPE QUESTION
    # ------------------------------------------------------------
    if question_type == "type":
        # Choose one of the Pokémon's actual types as the correct answer
        correct = random.choice(pokemon["types"])

        # Build a list of ALL types across the dataset
        all_types = sorted({t for types in df["types"] for t in types})

        # Pick 3 wrong types that are NOT the correct one
        wrong = random.sample([t for t in all_types if t != correct], 3)

        # Build the question text
        question = f"What is one of the types of {pokemon['name'].title()}?"

        # Combine correct + wrong answers
        options = [correct] + wrong

    # ------------------------------------------------------------
    # 3B. ABILITY QUESTION
    # ------------------------------------------------------------
    elif question_type == "ability":
        # Choose one of the Pokémon's abilities as the correct answer
        correct = random.choice(pokemon["abilities"])

        # Build a list of ALL abilities across the dataset
        all_abilities = sorted({a for abilities in df["abilities"] for a in abilities})

        # Pick 3 wrong abilities
        wrong = random.sample([a for a in all_abilities if a != correct], 3)

        # Build the question text
        question = f"Which of the following is an ability of {pokemon['name'].title()}?"

        # Combine correct + wrong answers
        options = [correct] + wrong

    # ------------------------------------------------------------
    # 3C. STAT QUESTION
    # ------------------------------------------------------------
    elif question_type == "stat":
        # Pick a random stat name (e.g., "hp", "attack", "speed")
        stat_name = random.choice(list(pokemon["stats"].keys()))

        # Correct answer is the Pokémon's actual stat value
        correct = pokemon["stats"][stat_name]

        # Generate 3 wrong answers by adding/subtracting random values
        wrong = [
            correct + random.randint(-20, -5),   # slightly lower
            correct + random.randint(5, 20),     # slightly higher
            correct + random.randint(21, 40)     # much higher
        ]

        # Build the question text
        question = f"What is {pokemon['name'].title()}'s {stat_name} stat?"

        # Combine correct + wrong answers
        options = [correct] + wrong

    # ------------------------------------------------------------
    # 4. Shuffle the answer options so the correct one isn't always first
    # ------------------------------------------------------------
    random.shuffle(options)

    # ------------------------------------------------------------
    # 5. Return everything needed for the quiz
    # ------------------------------------------------------------
    return question, options, correct


In [ ]:
def run_quiz(df, num_questions=5):
    """
    Runs an interactive Pokémon quiz in the console.

    Parameters:
        df (DataFrame): The Pokémon dataset.
        num_questions (int): How many questions to ask.

    The function:
      - generates a question
      - displays multiple‑choice options
      - accepts user input
      - checks correctness
      - tracks score
    """
    
    score = 0  # Keeps track of how many answers the user gets right

    # ------------------------------------------------------------
    # Loop through the desired number of questions
    # ------------------------------------------------------------
    for i in range(num_questions):

        # Generate a random quiz question
        question, options, correct = generate_question(df)

        # Display the question number and text
        print(f"\nQuestion {i+1}: {question}")

        # Display the multiple‑choice options (numbered 1–4)
        for idx, opt in enumerate(options):
            print(f"{idx+1}. {opt}")

        # ------------------------------------------------------------
        # Get the user's answer
        # ------------------------------------------------------------
        # input() returns a string, so convert to int
        answer = int(input("Your answer: "))

        # ------------------------------------------------------------
        # Check if the selected option matches the correct answer
        # ------------------------------------------------------------
        if options[answer - 1] == correct:
            print("Correct!")
            score += 1  # Increase score for correct answer
        else:
            print(f"Incorrect. The correct answer was: {correct}")

    # ------------------------------------------------------------
    # After all questions, show the final score
    # ------------------------------------------------------------
    print(f"\nFinal score: {score}/{num_questions}")


In [12]:
run_quiz(df, num_questions=10)



Question 1: What is Sawk's attack stat?
1. 115
2. 158
3. 140
4. 125
Incorrect. The correct answer was: 125

Question 2: What is Mightyena's special-attack stat?
1. 69
2. 42
3. 60
4. 88
Incorrect. The correct answer was: 60

Question 3: What is one of the types of Quilladin?
1. ghost
2. steel
3. fighting
4. grass
Incorrect. The correct answer was: grass

Question 4: What is Jigglypuff's attack stat?
1. 28
2. 45
3. 75
4. 56
Incorrect. The correct answer was: 45

Question 5: What is one of the types of Aipom?
1. ghost
2. electric
3. normal
4. water
Incorrect. The correct answer was: normal

Question 6: Which of the following is an ability of Pikachu-Gmax?
1. hyper-cutter
2. curious-medicine
3. static
4. effect-spore
Incorrect. The correct answer was: static

Question 7: What is one of the types of Celebi?
1. grass
2. psychic
3. fighting
4. water
Incorrect. The correct answer was: psychic

Question 8: Which of the following is an ability of Meowscarada?
1. liquid-ooze
2. long-reach
3. sim


Further possibilities - by Microsoft Copilot
🧠 1. Data Analysis & Feature Engineering
Perfect if you want to flex your pandas + data‑science muscles.

Ideas:
Flatten the stats into separate columns (HP, attack, defense, etc.)

One‑hot encode types (fire, water, grass…)

Count abilities and see which are most common

Calculate BMI‑style metrics (weight/height ratios)

Cluster Pokémon by stats using k‑means

Find outliers (heaviest, fastest, highest XP)

This is basically a fun dataset for practising real DS workflows.

📊 2. Visualisation Projects
Turn your DataFrame into charts.

Ideas:
Bar chart of average stats by type

Scatter plot of height vs weight

Heatmap of stat correlations

Distribution of base experience

Radar charts for individual Pokémon

This is great practice for matplotlib, seaborn, or plotly.

🧩 3. API Exploration & Expansion
You’ve only touched one endpoint — the API has loads more.

Explore:
/type → list Pokémon by type

/ability → see which Pokémon share abilities

/evolution-chain → build evolution trees

/species → get habitat, color, shape, growth rate

/moves → list all moves a Pokémon can learn

You can merge these into your main DataFrame.

🏗️ 4. Build Something Interactive
Turn your data into a mini‑application.

Ideas:
A Pokédex CLI

python pokedex.py pikachu → prints stats

A Flask or FastAPI web app

A search tool with filters (type, stat range, etc.)

A Discord bot that returns Pokémon info

This is great for portfolio projects.

🧬 5. Machine Learning Projects
Pokémon data is surprisingly good for ML practice.

Ideas:
Predict a Pokémon’s type from its stats

Predict base experience from height/weight

Cluster Pokémon into archetypes

Build a battle simulator using stats as features

You can even try neural networks if you want to go wild.

🧠 6. Data Cleaning & Normalisation
If you want to practise real‑world wrangling:

Flatten nested JSON

Normalise stats into separate tables

Build relational tables (Pokémon, types, abilities, stats)

Store everything in SQLite or PostgreSQL

This is excellent practice for data engineering.

🎮 7. Fun / Creative Projects
Because why not?

Generate random teams - done - To be rethought and adjusted later

Find the strongest team by type coverage

Build a Pokémon quiz generator

Create a Pokémon recommender system

Analyse which Pokémon would be “meta” in a hypothetical game